Dependencies

In [ ]:
!pip install boto3 faiss-cpu langchain pypdf

Imports & Bedrock Client Setup

In [ ]:

import boto3, json
import faiss
import numpy as np
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Bedrock client
session = boto3.Session(region_name="us-east-1")  # change region if needed
bedrock = session.client("bedrock-runtime")


Load PDF

In [ ]:
def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

pdf_path = "Bank_of_America_Service_Agreement.pdf"  # update with your file
document_text = load_pdf(pdf_path)
print("Document length (chars):", len(document_text))

Chunk Text

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

chunks = splitter.split_text(document_text)
print(f"Total chunks created: {len(chunks)}")
print(chunks[0][:500])  # preview first chunk


Generate Embeddings (Titan via Bedrock)

In [ ]:
EMBED_MODEL = "amazon.titan-embed-text-v1"

def embed_text(text):
    response = bedrock.invoke_model(
        modelId=EMBED_MODEL,
        contentType="application/json",
        accept="application/json",
        body=json.dumps({"inputText": text})
    )
    result = json.loads(response["body"].read().decode("utf-8"))
    return np.array(result["embedding"], dtype=np.float32)

# Test embedding
test_vector = embed_text("Hello World!")
print("Embedding length:", len(test_vector))


Build FAISS Index

In [ ]:
dimension = len(embed_text("test"))
index = faiss.IndexFlatL2(dimension)

embeddings = [embed_text(chunk) for chunk in chunks]
index.add(np.vstack(embeddings))

print("FAISS index built. Total vectors:", index.ntotal)


Define RAG Query Function

In [ ]:
LLM_MODEL = "anthropic.claude-3-sonnet-20240229-v1:0"

def rag_query(question, top_k=3):
    q_emb = embed_text(question).reshape(1, -1)
    _, I = index.search(q_emb, top_k)
    retrieved = [chunks[i] for i in I[0]]
    context = "\n\n".join(retrieved)
    
    prompt = f"""
    You are a helpful assistant. Answer based ONLY on the Bank of America Online Banking Service Agreement.
    If unsure, say "I don’t know."

    Context:
    {context}

    Question: {question}
    Answer:
    """
    
    response = bedrock.invoke_model(
        modelId=LLM_MODEL,
        contentType="application/json",
        accept="application/json",
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 300,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    return response["body"].read().decode("utf-8")


Sample Queries

In [ ]:
print(rag_query("What is the Zelle transfer limit for new users?"))
print(rag_query("How do I cancel a scheduled bill payment?"))
print(rag_query("What is the cut-off time for domestic wire transfers?"))
